In [1]:
import pandas as pd 
import numpy as np
import os, sys
import scipy.stats as st
from scipy.stats import ttest_ind
import datetime
import pathlib

confidence_level = 0.95
radius = 0.5
numbers_until_converge = np.zeros(9)

def isvalid(df):
    # cond1 = (df["usertimes"] > df["systemtimes"]).all()
    cond0 = df.query("videos == '5.mp4'")["scores"].item() >= df.query("videos == '6.mp4'")["scores"].item()

    cond2 = df.query("videos == '1.mp4'")["scores"].item() == df["scores"].max()
    cond3 = df.query("videos == '2.mp4'")["scores"].item() == df["scores"].min()
    cond4 = df.query("videos == '1.mp4'")["sanityscore"].item() == 2
    cond5 = df.query("videos == '2.mp4'")["sanityscore"].item() == 5
    #cond6 = (df.query("videos == '3.mp4'")["sanityscore"].item() == 5) or (df.query("videos == '3.mp4'")["sanityscore"].item() == 4) 
    return cond0 and cond2 and cond3 and cond4 and cond5 #and cond6
    #return True

def process_one_file(filename):
    with open(filename, "r") as fin:
        lines = [l.strip("\n") for l in fin]
    scores = [int(v) for v in lines[0].split(',')]
    videos = [f"{v}.mp4" for v in lines[1].split(",")]
    moved_scores = []
    for i in [int(v) for v in lines[1].split(',')]:
        moved_scores.append(scores[i-1])
    scores = moved_scores
    # videos = [f"{v}.mp4" for v in range(1, 10)]
    videos = videos[:len(scores)]
    usertimes = [int(v) for v in lines[2].split(',')]
    systemtimes = [int(v) for v in lines[3].split(',')]
    userid = lines[4]
    sanityscore = [int(v) for v in lines[9].split(',')]
    end_t = os.path.getmtime(filename)
    timestamps = []
    for i in range(len(usertimes)):
        sum = 0
        for j in range(i+1, len(usertimes)):
            sum+=usertimes[j]
        sum /= 1000
        timestamps.append(end_t - sum -1.68*10**9)

    print(timestamps, usertimes)
    df = pd.DataFrame()

    df["scores"] = scores
    df["videos"] = videos
    df["usertimes"] = usertimes
    df["systemtimes"] = systemtimes
    df["userid"] = userid
    df["sanityscore"] = sanityscore
    df["end_time"] = timestamps
    print(df["end_time"])
    

    if isvalid(df):
        return df
    else:
        return None



In [2]:
#Set files
filenames = []
# os.listdir("/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/results_ow1/")

for filepath in pathlib.Path("/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/results_ow1/").glob('**/*'):
    filenames.append(str(filepath))



In [3]:
# Load files
dfs = []
for file in filenames:
    if "txt" in file:
        df = process_one_file(file)
        try:
            df = process_one_file(file)
            if df is None:
                print(file, "is invalid!")
            else:
                print(df)
                dfs.append(df)
        except:
            pass
final_df = pd.concat(dfs)

groupedby = final_df[["videos", "scores"]].groupby("videos")


print(final_df[["videos", "scores"]].groupby("videos").mean().reset_index(drop=True))

        

[2357884.9111201763, 2357903.585120201, 2357923.2601201534, 2357942.87912035, 2357961.520120144, 2357980.160120249] [45687, 18674, 19675, 19619, 18641, 18640]
0    2.357885e+06
1    2.357904e+06
2    2.357923e+06
3    2.357943e+06
4    2.357962e+06
5    2.357980e+06
Name: end_time, dtype: float64
[2357884.9111201763, 2357903.585120201, 2357923.2601201534, 2357942.87912035, 2357961.520120144, 2357980.160120249] [45687, 18674, 19675, 19619, 18641, 18640]
0    2.357885e+06
1    2.357904e+06
2    2.357923e+06
3    2.357943e+06
4    2.357962e+06
5    2.357980e+06
Name: end_time, dtype: float64
   scores videos  usertimes  systemtimes          userid  sanityscore  \
0       5  1.mp4      45687        14188  A27W025UEXS1G0            2   
1       1  2.mp4      18674        17794  A27W025UEXS1G0            5   
2       2  5.mp4      19675         8049  A27W025UEXS1G0            4   
3       3  3.mp4      19619        12219  A27W025UEXS1G0            4   
4       2  6.mp4      18641        4232

In [4]:
sorted_df = pd.concat(dfs).sort_values(by = ["end_time"])
sorted_df.to_csv("/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/temp.csv")
print(len(dfs))

27


In [5]:
def get_confidence(data):
    return st.t.interval(alpha=confidence_level, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) 


In [7]:
flags = np.zeros(7)
counters = np.zeros(7)
num_until_confident_not_equal = np.zeros(7)
stored_numbers = []
sum = 0

for i in range(7):
    stored_numbers.append([])

for index, row in sorted_df.iterrows():
    # print(row)
    id = int(row["videos"].split('.')[0])
    quality = int(row["scores"])
    stored_numbers[id].append(quality)
    
    confidence = get_confidence(stored_numbers[id])
    if (id == 3 or id == 4):
        # print(id, stored_numbers[id], stored_numbers[5], ttest_ind(stored_numbers[5], stored_numbers[id], equal_var=False))
        ttest = ttest_ind(stored_numbers[5], stored_numbers[id], equal_var=False)
        
        if(abs(ttest.statistic) > st.t.ppf(q=1-.05, df = len(stored_numbers[5]) + len(stored_numbers[id])) and num_until_confident_not_equal[id]==0 and len(stored_numbers[5]) + len(stored_numbers[id])>15):
            num_until_confident_not_equal[id] = len(stored_numbers[id])

    print(id, st.sem(stored_numbers[id]))
    if (np.mean(stored_numbers[id]) - confidence[0]<radius and numbers_until_converge[id-2] ==0):
        sum+=1
        numbers_until_converge[id-2] = len(stored_numbers[id])
        print(id, stored_numbers[id],confidence[0])

loss60_numbers = stored_numbers[5]
print("number of tests until confident they are not equal for 1st stall: ", num_until_confident_not_equal[3])
print("number of tests until confident they are not equal for 2nd stall: ", num_until_confident_not_equal[4])
print("with enough confidence: ",sum)

nan
nan
nan
nan
nan
nan
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.5
0.0
0.0
0.33333333333333337
0.25
0.33333333333333337
0.25
0.33333333333333337
0.28867513459481287
0.0
0.25
0.0
0.0
0.19999999999999996
0.0
0.19999999999999998
0.0
0.24494897427831777
0.0
0.0
0.0
0.19999999999999996
0.16666666666666666
0.42857142857142855
0.0
0.22360679774997896
0.16666666666666666
0.14285714285714285
0.25819888974716115
0.2020305089104421
0.37499999999999994
0.16366341767699427
0.3595159254890833
0.2630521404045756
0.3238992347717331
0.0
0.0
0.14698618394803278
0.3333333333333333
0.24216105241892627
0.28867513459481287
0.0
0.0
0.0
0.29059326290271154
0.15275252316519466
0.0
0.30550504633038933
0.2816715160878121
0.26666666666666666
0.2439346884545225
0.26347957720344434
0.14083575804390605
0.0
0.0
0.0
0.0
0.2599047999758855
0.1305582419667734
0.12162606385262997
0.24099960182707375
0.22890825651118377
0.21299035545943784
0.0
0.24325212770525995
0.0
0.23076923076923075
0.22060285604268937
0.20203050891044216
0.2482

/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_2081631/2797391819.py:2: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  return st.t.interval(alpha=confidence_level, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-pac

In [7]:
#Set files
filenames = []
# os.listdir("/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/results_ow1/")

for filepath in pathlib.Path("/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/results_ow2/").glob('**/*'):
    filenames.append(str(filepath))



In [8]:
# Load files
dfs = []
for file in filenames:
    if "txt" in file:
        df = process_one_file(file)
        try:
            df = process_one_file(file)
            if df is None:
                print(file, "is invalid!")
            else:
                print(df)
                dfs.append(df)
        except:
            pass
        

[2701317.916673422, 2701339.5346734524, 2701360.8096735477, 2701379.66067338, 2701399.0796735287, 2701419.4326734543] [51262, 21618, 21275, 18851, 19419, 20353]
0    2.701318e+06
1    2.701340e+06
2    2.701361e+06
3    2.701380e+06
4    2.701399e+06
5    2.701419e+06
Name: end_time, dtype: float64
[2701317.916673422, 2701339.5346734524, 2701360.8096735477, 2701379.66067338, 2701399.0796735287, 2701419.4326734543] [51262, 21618, 21275, 18851, 19419, 20353]
0    2.701318e+06
1    2.701340e+06
2    2.701361e+06
3    2.701380e+06
4    2.701399e+06
5    2.701419e+06
Name: end_time, dtype: float64
/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/results_ow2/A207IHY6GERCFO.txt is invalid!
[2701323.8316733837, 2701342.9316735268, 2701363.6076734066, 2701382.691673517, 2701401.3256733418, 2701419.4326734543] [108620, 19100, 20676, 19084, 18634, 18107]
0    2.701324e+06
1    2.701343e+06
2    2.701364e+06
3    2.701383e+06
4    2.701401e+06
5    2.701419e+06
Name: end_time, dtype: float6

In [9]:
sorted_df = pd.concat(dfs).sort_values(by = ["end_time"])
sorted_df.to_csv("/dataheart/hanchen/qoe_platform/QoE_experiments_2023apr/temp.csv")
print(len(dfs))

25


In [10]:
def get_confidence(data):
    return st.t.interval(alpha=confidence_level, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) 


In [11]:
flags = np.zeros(7)
counters = np.zeros(7)
num_until_confident_not_equal = np.zeros(7)
stored_numbers = []
sum = 0

for i in range(7):
    stored_numbers.append([])

for index, row in sorted_df.iterrows():
    # print(row)
    id = int(row["videos"].split('.')[0])
    quality = int(row["scores"])
    stored_numbers[id].append(quality)
    confidence = get_confidence(stored_numbers[id])
    if (id == 3 or id == 4):
        ttest = ttest_ind(loss60_numbers, stored_numbers[id], equal_var=False)
        if (len(stored_numbers[id]) > 15):
            print(id, ttest.pvalue)        
        if(abs(ttest.statistic) > st.t.ppf(q=1-.05, df = len(loss60_numbers) + len(stored_numbers[id])) and num_until_confident_not_equal[id]==0 and  len(stored_numbers[id])>15):
            print(np.mean(stored_numbers[id]), np.mean(loss60_numbers))
            num_until_confident_not_equal[id] = len(stored_numbers[id])

    print(st.sem(stored_numbers[id]))
    if (np.mean(stored_numbers[id]) - confidence[0]<radius and numbers_until_converge[id+2] ==0):
        sum+=1
        numbers_until_converge[id+2] = len(stored_numbers[id])
        print(id, stored_numbers[id],confidence[0])


print("number of tests until confident they are not equal for 1st stall: ", num_until_confident_not_equal[3])
print("number of tests until confident they are not equal for 2nd stall: ", num_until_confident_not_equal[4])
print("with enough confidence: ",sum)

3 [4, 3, 3, 4, 4, 4, 4] 3.2630071980948676
3 0.2285316791970559
5 [4, 4, 5, 5, 5, 4, 2, 4, 4, 4, 4, 4, 3] 3.5065960542545023
6 [4, 4, 3, 3, 3, 3, 2, 4, 3, 2, 1, 4, 3, 3, 2] 2.443948222016301
4 [4, 4, 3, 3, 2, 4, 3, 3, 2, 4, 3, 3] 2.710636069846852
3 0.17343769988665095
3 0.2989395763315784
3 0.3216100164439483
4 0.660025349655055
3 0.6175704319924261
4 0.8351890296864923
4 0.6581349465246262
4 0.6487922794618604
3 0.7737894402955541
4 0.9656432056131088
3 0.7922065647136729
4 0.804558025588687
4 0.7928922549046679
3 0.8101945407427893
4 0.7818758174126781
4 0.7714769581903498
3 0.7059263362996605
4 0.6410481742151835
3 0.7242609288294724
number of tests until confident they are not equal for 1st stall:  0.0
number of tests until confident they are not equal for 2nd stall:  0.0
with enough confidence:  4


/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_1961987/2797391819.py:2: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  return st.t.interval(alpha=confidence_level, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/dataheart/hanchen/anaconda3/envs/ae-env/lib/python3.8/site-pac

In [12]:
print(numbers_until_converge)

[ 0.  6.  7. 14.  6.  7. 12. 13. 15.]
